In [1]:
import pandas as pd


Section 1: Data Preparation and Exploration

Group and count the number of users by 'utm_source' and rename the column

In [2]:
ad_clicks = pd.read_csv('ad_clicks.csv', delimiter=',')
print(ad_clicks.head(10))
sorted_by_view = ad_clicks.groupby('utm_source').user_id.count().reset_index()
sorted_by_view = sorted_by_view.rename(columns={'user_id': "views"})
print(sorted_by_view)

                                user_id utm_source            day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook     1 - Monday   
6  0153d85b-7660-4c39-92eb-1e1acd023280     google   4 - Thursday   
7  01555297-d6e6-49ae-aeba-1b196fdbb09f     google  3 - Wednesday   
8  018cea61-19ea-4119-895b-1a4309ccb148      email     1 - Monday   
9  01a210c3-fde0-4e6f-8efd-4f0e38730ae6      email    2 - Tuesday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN          

Create a new column 'is_click' based on the presence of 'ad_click_timestamp'


In [3]:
ad_clicks['is_click'] = ~ ad_clicks.ad_click_timestamp.isnull() 
print(ad_clicks.head(10))

                                user_id utm_source            day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook     1 - Monday   
6  0153d85b-7660-4c39-92eb-1e1acd023280     google   4 - Thursday   
7  01555297-d6e6-49ae-aeba-1b196fdbb09f     google  3 - Wednesday   
8  018cea61-19ea-4119-895b-1a4309ccb148      email     1 - Monday   
9  01a210c3-fde0-4e6f-8efd-4f0e38730ae6      email    2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  
3                NaN            

Section 2: Click Analysis by Source

Group and count the number of users by 'utm_source' and 'is_click'

In [4]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
print(clicks_by_source.head(5))


  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441


 Pivot the data to have 'is_click' as columns and 'utm_source' as index

In [5]:
clicks_pivot = clicks_by_source.pivot(
columns = 'is_click',
index = 'utm_source',
values = 'user_id'
).reset_index()

Calculate the 'percent_clicked' for each source

In [6]:
clicks_pivot['percent_clicked'] =  ((clicks_pivot[True] / (clicks_pivot[True] + 
clicks_pivot[False])) * 100).round()
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80             31.0
1          facebook    324   180             36.0
2            google    441   239             35.0
3           twitter    149    66             31.0


Section 3: Experimental Group Analysis

Count the number of users by 'experimental_group'

In [7]:
exp_group_count = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print(exp_group_count)


  experimental_group  user_id
0                  A      827
1                  B      827


Group and count the number of users by 'is_click' and 'experimental_group'

In [8]:
AB_click_perc = ad_clicks.groupby(['is_click', 'experimental_group']).user_id.count().reset_index()
print(AB_click_perc)

   is_click experimental_group  user_id
0     False                  A      517
1     False                  B      572
2      True                  A      310
3      True                  B      255


Pivot the data to have 'is_click' as columns and 'experimental_group' as index

In [9]:
AB_click_perc_pivot = AB_click_perc.pivot(
columns = 'is_click',
index = 'experimental_group',
values = 'user_id'
)
print(AB_click_perc_pivot)

is_click            False  True 
experimental_group              
A                     517    310
B                     572    255


Section 4: Click Analysis by Day (A and B Groups)

In [10]:
a_clicks = ad_clicks[
   ad_clicks.experimental_group
   == 'A'].reset_index()
# print(a_clicks)
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B'].reset_index()
# print(b_clicks)

Group and count the number of users by 'day' and 'is_click' for groups

In [11]:
a_clicks_pivot = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index().pivot(
columns = 'is_click',
index = 'day',
values = 'user_id'
)
b_clicks_pivot= b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index().pivot(
columns = 'is_click',
index = 'day',
values = 'user_id'
)
print(a_clicks_pivot)

is_click       False  True 
day                        
1 - Monday        70     43
2 - Tuesday       76     43
3 - Wednesday     86     38
4 - Thursday      69     47
5 - Friday        77     51
6 - Saturday      73     45
7 - Sunday        66     43


Calculate the 'percentage_clicked' for each day in both groups

In [12]:
a_clicks_pivot['percentage_clicked'] = (a_clicks_pivot[True] / (a_clicks_pivot[True] + 
a_clicks_pivot[False]))
b_clicks_pivot['percentage_clicked'] = (b_clicks_pivot[True] / (b_clicks_pivot[True] + 
b_clicks_pivot[False]))

print(a_clicks_pivot)
print(b_clicks_pivot)

is_click       False  True  percentage_clicked
day                                           
1 - Monday        70    43            0.380531
2 - Tuesday       76    43            0.361345
3 - Wednesday     86    38            0.306452
4 - Thursday      69    47            0.405172
5 - Friday        77    51            0.398438
6 - Saturday      73    45            0.381356
7 - Sunday        66    43            0.394495
is_click       False  True  percentage_clicked
day                                           
1 - Monday        81    32            0.283186
2 - Tuesday       74    45            0.378151
3 - Wednesday     89    35            0.282258
4 - Thursday      87    29            0.250000
5 - Friday        90    38            0.296875
6 - Saturday      76    42            0.355932
7 - Sunday        75    34            0.311927
